# AIS Example

Example taken from https://pymeos.readthedocs.io/en/latest/src/examples/AIS.html

In [ ]:
pip install tqdm
pip install matplotlib
pip install pandas
pip install contextily

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
from pymeos import *
from tqdm.notebook import tqdm
import contextily as cx

tqdm.pandas()

pymeos_initialize()

## Reading from File ([MEOS Example](https://libmeos.org/tutorialprograms/meos_read_ais/))
In this section, AIS data is read from a CSV file. We then use the read information to recreate the trajectories of the ships.



First, let's read the CSV file using pandas. We will read only the columns we're interested in, and remove the observations that are either erroneous or not in Denmark.

In [ ]:
%%time
#wget https://web.ais.dk/aisdata/aisdk-2023-06-01.zip
ais = pd.read_csv(
    "aisdk-2023-06-01.zip",
    usecols=["# Timestamp", "MMSI", "Latitude", "Longitude", "SOG"],
)
ais.columns = ["t", "mmsi", "lat", "lon", "sog"]
ais = ais[ais["t"] != 0]
ais["t"] = pd.to_datetime(ais["t"], format='%d/%m/%Y %H:%M:%S')
ais = ais[ais["mmsi"] != 0]
ais = ais.drop_duplicates(["t", "mmsi"])
ais = ais[(ais["lat"] >= 40.18) & (ais["lat"] <= 84.17)]
ais = ais[(ais["lon"] >= -16.1) & (ais["lon"] <= 32.88)]
ais = ais[(ais["sog"] >= 0) & (ais["sog"] <= 1022)]
ais.dropna()
ais.head()

Now, we will create the PyMEOS object representing the position and the SOG.

In [ ]:
ais["instant"] = ais.progress_apply(
    lambda row: TGeomPointInst(point=(row["lon"], row["lat"]), timestamp=row["t"]),
    axis=1,
)
ais["sog"] = ais.progress_apply(
    lambda row: TFloatInst(value=row["sog"], timestamp=row["t"]), axis=1
)

In [ ]:
ais.drop(["lat", "lon"], axis=1, inplace=True)
ais.head()

## Assembling Trips ([MEOS Example](https://libmeos.org/tutorialprograms/meos_assemble_ais/))

Now, we will create the trajectory (TGeogPointSeq) and the SOG evolution (TFloatSeq) for every ship (identified by the mmsi) using the instants we have created.

In [ ]:
%%time
trajectories = (
    ais.groupby("mmsi")
    .aggregate(
        {
            "instant": lambda x: TGeomPointSeq.from_instants(x, upper_inc=True),
            "sog": lambda x: TFloatSeq.from_instants(x, upper_inc=True),
        }
    )
    .rename({"instant": "trajectory"}, axis=1)
)
trajectories["distance"] = trajectories["trajectory"].apply(lambda t: t.length())
trajectories.head()

Here we can see that PyMEOS has been able to reduce the number of points stored (and thus memory used) without losing any information.

We can visualize the trajectories and the SOG evolutions by plotting them. We will plot only 100 of the trajectories.

## Storing in MobilityDB ([MEOS Example](https://libmeos.org/tutorialprograms/meos_store_ais/))

Now we will store the temporal objects we have created in MobilityDB.
To connect to MobilityDB (PostgreSQL), `psycopg` is used. However, `asyncpg` and `psycopg2` are also supported.

First, set up the connection parameters. Change any of the following values according to your configuration. If you are using the docker image, you don't need to change anything (except maybe the port).

In [ ]:
from pymeos.db.psycopg import MobilityDB

host = "localhost"
port = 5432
db = "mobilitydb"
user = ""
password = ""

connection = MobilityDB.connect(
    host=host, port=port, dbname=db, user=user, password=password
)
cursor = connection.cursor()

Now, we will create the table where we will write our objects.

In [ ]:
cursor.execute("DROP TABLE IF EXISTS public.PyMEOS_demo;")
cursor.execute(
    "CREATE TABLE public.PyMEOS_demo"
    "(MMSI integer, trajectory public.tgeompoint, SOG public.tfloat);"
)
connection.commit()

Let's insert now the rows of the DataFrame into the MobilityDB table. First, we'll create a SQL INSERT query with all the values, and then we will execute it in MobilityDB.

In [ ]:
query = "INSERT INTO public.PyMEOS_demo(MMSI, trajectory, SOG) VALUES"
for mmsi, row in trajectories.iterrows():
    query += f"\n({mmsi}, '{row.trajectory}', '{row.sog}'),"
query = query[:-1] + ";"

In [ ]:
%%time
cursor.execute(query)
connection.commit()

Let's check how many rows we just added

In [ ]:
cursor.execute("SELECT COUNT(*) FROM public.PyMEOS_demo;")
cursor.fetchone()[0]

Now, we will read one of the records that we just wrote

In [ ]:
cursor.execute("SELECT * FROM public.PyMEOS_demo WHERE MMSI = 308654000 ;")
mmsi, trajectory, sog = cursor.fetchone()

In [ ]:
trajectory

Finally, let's close the connection and wrap everything up

In [ ]:
connection.commit()
cursor.close()
pymeos_finalize()